In [1]:
import requests
import requests
import concurrent.futures
from typing import List, Tuple, Optional
import time
import os
import tqdm
import json
import pandas as pd
from parse_wordjson import JsonParse
from concurrent.futures import ThreadPoolExecutor, as_completed

In [21]:
def sub_query(url, query, model_name):
    if query == "-1":
        return "-1", None

    session = requests.Session()
    json_data = {
            "model": model_name,
            "messages": [{"role": "system", "content": "You are a helpful assistant"},
                         {"role": "user", "content": query}],
        }
    
    config_dict = {
        "n": 1,
        "temperature":1,
        "top_p":0.7,
        "top_k":5,
        # # "use_beam_search": True,
        # # "num_beams": 5,
        "length_penalty":1,
        "repetition_penalty": 1.05,
        # "min_tokens": 200,
        "max_tokens": 4000,
        "frequency_penalty":0.5
    }
    json_data.update(config_dict)

    max_retries = 6
    for attempt in range(max_retries):
        try:
            response = session.post(url, json=json_data)
            response.raise_for_status()
            res_text = response.json()
            final_response = (
                res_text.get("choices", [{}])[0].get("message", {}).get("content")
            )
            # count += 1
            # print(count)
            print("has done")
            return final_response
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt+1} failed, retrying...")
                print(len(json_data["messages"][1]['content']))
                continue
            else:
                print(f"Final attempt failed with error: {e}")
                return None

    return None

In [22]:
ip_address, port = "10.96.202.31", 8000
url = "http://%s:%d/v1/chat/completions" % (ip_address, port)
result = sub_query(url, "你好", "Yi-34B-Chat_20240123_223712/")
result

has done


'你好！有什么可以帮你的吗？'

In [3]:
jp = JsonParse()
path = pd.ExcelFile("数据查找_结构化较差文章-4.xlsx")
df_old = pd.DataFrame()
for name in path.sheet_names[2:]:
    df_old = pd.concat([df_old, pd.read_excel(path, sheet_name=name)])
df_old['docid'] = range(len(df_old))
all_data = []
repeat_source = set()
for idx in range(1200):
    try:
        with open(f"download/{idx}-{idx}/{idx}.json")as f:
            bdjson = json.load(f)
        source = jp.parse_word_v2(bdjson['word'])
        source = "\n".join([list(p.values())[0] for p in source.values() if isinstance(list(p.values())[0], str)])
        if source.strip()[:35] in repeat_source:
            continue
        else:
            repeat_source.add(source.strip()[:35])
        all_data.append({
            "docid": idx,
            "source": source,
            "url": list(df_old['url'])[idx]
        })
    except:
        print(idx)
        continue
len(all_data)

  File "/home/users/hanwanpeng/workshop/bad_struct_source_opt/parse_wordjson.py", line 309, in v2_parse_table
    table_html = self.create_html_table(table_data)
  File "/home/users/hanwanpeng/workshop/bad_struct_source_opt/parse_wordjson.py", line 257, in create_html_table
    if colspan[row_inx][col_inx] > 1 and rowspan[row_inx][col_inx] > 1:
IndexError: list index out of range

  File "/home/users/hanwanpeng/workshop/bad_struct_source_opt/parse_wordjson.py", line 309, in v2_parse_table
    table_html = self.create_html_table(table_data)
  File "/home/users/hanwanpeng/workshop/bad_struct_source_opt/parse_wordjson.py", line 257, in create_html_table
    if colspan[row_inx][col_inx] > 1 and rowspan[row_inx][col_inx] > 1:
IndexError: list index out of range

  File "/home/users/hanwanpeng/workshop/bad_struct_source_opt/parse_wordjson.py", line 309, in v2_parse_table
    table_html = self.create_html_table(table_data)
  File "/home/users/hanwanpeng/workshop/bad_struct_source_opt/parse_wo

1062
1063
1068
1069
1073
1074
1075
1076
1077
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199


981

In [4]:
prompt = """{}

【任务描述】
抽取以上文章的大纲，用markdown格式输出。

【具体要求】
1、大纲中不应包含详情，应仅使用一级、二级、三级和四级标题，分别以 "# ", "## ", "### ", "#### " 开头。
2、请确保二级标题至少有两个，并且如果某个二级标题下包含三级标题，那么这些三级标题的数量必须大于一个。
3、对于原文结构化较差的文章，对原文进行概括、总结，保证大纲的逻辑清晰、层级分明。"""

df = pd.DataFrame(all_data)
df = df[df['docid']>=799]
df.shape
df = df[df['source'].str.len()>650]
df.shape
df['all_text'] = df['source'].apply(lambda x: prompt.format(x))
# df.to_excel("0425_跑对比数据.xlsx", index=False)

In [8]:
def fetch_outline(url: str, text: str, model_name: str) -> Tuple[str, Optional[str]]:
    """ 包装函数，用于从sub_query返回结果 """
    return sub_query(url, text, model_name)

count = 0
ip_address, port = "10.96.202.40", 8000
# ip_address, port = "10.96.202.13", 8000
url = "http://%s:%d/v1/chat/completions" % (ip_address, port)
novel_name = "TEST"
model_name = "Qwen1.5-72B-Chat/"
all_texts = df['all_text'].tolist()  # 从DataFrame获取所有请求文本

results = []

with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(tqdm.tqdm(executor.map(fetch_outline, [url]*len(all_texts), all_texts, [model_name]*len(all_texts)), total=len(all_texts)))

df['result3'] = results

  0%|          | 0/266 [00:00<?, ?it/s]

has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
h

  0%|          | 1/266 [01:47<7:56:59, 108.00s/it]

has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has donehas done

has done
has done
has done
has done
has done
has done
has donehas done
has done

has done
has done
has done
has done
has done


  2%|▏         | 5/266 [02:09<1:30:06, 20.72s/it] 

...

has done


 85%|████████▍ | 226/266 [02:35<00:11,  3.63it/s]

has done
has done
has done
has done


100%|██████████| 266/266 [02:47<00:00,  1.59it/s]

has done


In [10]:
df.head(1)
df.to_excel("0425_bad_struct_qwen_result.xlsx", index=False)